In [1]:
from datetime import datetime

In [2]:
try:
    from google.colab import drive
    drive.mount('/content/gdrive')
    import google.colab
    inColab = True
except ImportError:
    inColab = False

In [3]:
if inColab == True:
    !pip install -U pandas==2.2.2 numpy==2.0.2 scipy==1.14.1 accelerate==1.6.0 peft==0.15.2 bitsandbytes==0.45.5 transformers==4.51.3 trl==0.16.1 datasets==3.5.0 tensorboard==2.19.0

In [11]:
import os
import torch
from datasets import load_dataset

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer
from datetime import datetime
import huggingface_hub

torch.cuda.is_available()

if inColab == True:
    from google.colab import drive
    drive.mount("/content/gdrive")

# huggingface_hub.login()

"""# 1. set model and dataset

### ★★★ 수정 포인트 ★★★
"""
# base_model = "google/gemma-2b-it"
base_model = "google/gemma-3-4b-it"
dataset_name = "hyokwan/familicare_health_general_knowledge"
datasetCommon = load_dataset(dataset_name, split="train")

In [12]:
# datasetCommon.push_to_hub("hyokwan/senior_sleeping")

In [13]:
datasetCommon

datasetCommon[10]

{'instruction': '58세 남자가 3시간 전부터 시작된 토혈로 응급실에 내원하였다. 환자는 5년 전 간경화증으로 진단받아 항바이러스제를 복용 중이다. 활력징후는 혈압 115/80mmHg, 맥박 85회/분, 호흡 18회/분, 체온 36.5°C이다. 신체 진찰에서 양쪽 팔다리에 여러 개의 멍이 관찰되었다. 검사 결과는 다음과 같다. 이 환자에게 가장 적절한 치료는 무엇인가?  \n혈액검사: 혈색소 10.3g/dL, 백혈구 3,200/mm³, 혈소판 90,000/mm³, 프로트롬빈시간 42초 (참고치: 12.7~15.4초), 활성화부분트롬보플라스틴시간 68초 (참고치: 26.3~39.4초) \n\n1) 비타민 K 근육주사  \n2) VIII인자 정맥주사  \n3) 혈소판농축물 수혈  \n4) 농축적혈구 수혈  \n5) 신선냉동혈장 수혈',
 'input': '내과',
 'output': '5) 신선냉동혈장 수혈'}

In [14]:
# filtered_dataset = datasetCommon.filter(lambda x: x["input"] == "내과")

In [15]:
datasetCommon = datasetCommon.filter(lambda x: x["input"] == "내과")

In [16]:
datasetCommon = datasetCommon.select(range(min(100, len(datasetCommon))))

In [17]:
datasetCommon.shape

(100, 3)

In [18]:
datasetCommon[0]

def convert_to_gemma_format(dataset):
    instruction = dataset['instruction']
    input=dataset['input']
    output = dataset['output']
    text = f"<start_of_turn>user\n{instruction}\n{input}<end_of_turn>\n<start_of_turn>model\n{output}<end_of_turn>"

    #text = f"<start_of_turn>{instruction}<end_of_turn>{input}<start_of_turn>{output}<end_of_turn>"
    dataset["text"] = text
    return dataset

datasetCommon = datasetCommon.map(convert_to_gemma_format,remove_columns=['instruction', 'input', 'output'])

print(datasetCommon[0])

datasetCommon

datasetCommon[10]

"""# 2. Config efficient fine-tuning with low-rank adaptation."""
torch.cuda.get_device_capability()[0]

{'text': '<start_of_turn>user\n23세 여자가 3개월 전부터 기침을 한다며 내원했다. 기침은 밤에 누워 자려고 할 때 심해진다고 한다. 1년 전에도 같은 시기에 기침이 3개월 동안 지속되다가 저절로 호전된 병력이 있다. 콧물이나 인후부 불편감은 없으며, 비흡연자이고 복용 중인 약물도 없다. 신체검사에서 혈압 120/80mmHg, 맥박 78회/분, 호흡 18회/분, 체온 36.5°C로 측정되었다. 청진상 호흡음은 정상이었고, 가슴 X선 사진과 코곁굴 X선 사진에서도 이상 소견이 없었다. 폐기능검사 결과는 다음과 같다.  \n- 강제 폐활량(FVC): 정상 예측치의 91%  \n- 1초간 강제날숨유량(FEV1): 정상 예측치의 85%  \n- 1초간 강제날숨유량/강제폐활량(FEV1/FVC): 75%  \n\n이 환자에서 다음으로 시행해야 할 검사는 무엇인가?  \n1) 기관지내시경  \n2) 기관지 확장제 반응 검사  \n3) 가슴 컴퓨터단층촬영(CT) \n4) 메타콜린 기관지유발검사  \n5) 폐 확산능 검사\n내과<end_of_turn>\n<start_of_turn>model\n4) 메타콜린 기관지유발검사<end_of_turn>'}


8

In [ ]:
# 현재 사용 중인 GPU의 주요 아키텍처 버전을 반환 8버전 이상 시 bfloat16 활용
# NVIDIA Ampere 아키텍처 이상 시에만 처리
# 정확도를 위하여 float 16이 타도록 강제설정 8->10 ★ 향후 변경 필요
if torch.cuda.get_device_capability()[0] >= 8:
    # 고속 attention 메커니즘을 구현하는 라이브러리
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16


# BitsAndBytesConfig 객체활용 양자화 설정
quant_config = BitsAndBytesConfig(
    # 모델을 4비트 양자화하여 로드할지 여부 결정
    load_in_4bit=True,
    # 양자화 방법 (nf4: Non-Uniform Quantization, "nf4","fp16 등))
    bnb_4bit_quant_type="nf4",
    # (4비트 양자화 시 사용할 데이터 타입, "torch.float16, bfloat16, float32 등)
    bnb_4bit_compute_dtype=torch_dtype,
    # 이중 양자화 사용여부 (이중 양자화는 양자화 과정에서 정밀도 높이기 위해 활용, 대신 더 연산은 복잡)
    bnb_4bit_use_double_quant=False
)

"""# 3. Load Pre-trained Language Model"""

model = AutoModelForCausalLM.from_pretrained(
    # 불러올 모델 정의
    base_model,
    # 모델 양자화 설정값
    quantization_config=quant_config,
    # 모델의 레이어를 할당할 장치 ("":0 -> 전체 모델을 GPU 0에 할당, "auto"는 알아서, "{"layer_0":0, ... 형태로 레이어별 할당 가능)
    # device_map={"": 0}
    device_map="auto"
)
# 캐시 사용 여부 (모델 출력 매번 새로 계산)
# 새로운 데이터에 대한 계산 시 저장하는게 불필요함 또한 True 시 캐시 저장할 메모리 사용하여 메모리 사용량 증가함
model.config.use_cache = False
# 모델읜 pretraining tensor parallelism 설정 1인경우 병렬처리 안하고 단일장치 활용 2이상 시 병렬 처리를 분산 가능
# 병렬 처리 시 추가적인 메모리 오버헤드 발생 가능함
model.config.pretraining_tp = 1

"""# 4. Load Pre-trained Language Model Tokenizer"""

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)

# ★★★ 수정포인트 ★★★
# Load tokenizer ( tokenizer.model 파일 생성됨, 단, pip install sentencepiece 설치 <-- GGUF 파일 생성 시 필요)
# tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True, use_fast=False)

# Must add EOS_TOKEN at response last line
tokenizer.pad_token = tokenizer.eos_token

# ★수정 포인트!!! 기존 # tokenizer.padding_side = "right"
EOS_TOKEN = tokenizer.eos_token
def prompt_eos(sample):
    sample['text'] = sample['text']+EOS_TOKEN
    return sample
datasetCommon = datasetCommon.map(prompt_eos)

# datasetCommon[199]

import gc
# # Flush memory
# del trainer, model
gc.collect()
torch.cuda.empty_cache()



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [20]:
torch_dtype

torch.bfloat16

In [21]:
"""# 5. Config training parameter for LoRA (Parameter-Efficient Fine-Tuning (PEFT)

"""

peft_params = LoraConfig(
    # lora 방식의 튜닝에서 저차원 병렬 레이어 학습 ( 값이 크면 저차원 병렬 레이어 영향이 커지고 작으면 줄어듬, 기본 16, 1~128)
    lora_alpha=16,
    # 과적합 방지 (0.1에서 증가 시킴, 0.0~0.3)
    lora_dropout=0.1,
    # LoRA 저차원 공간의 차원 수 (값이 커지면 파라미터 공간 커지지만 계산비용 증가 4, 8, 16, 32, 64 로 튜닝, 4~128)
    r=64,
    # LoRA 모델의 바이어스 파라미터 적용여부 None = 미적용 ("none" or "all", "lora_only" 설정 가능)
    bias="none",
    # LoLA 모델의 작업 유형 (GPT와 같은 인과모델) ("CASUAL_LM": GPT, "SEQ_CLASSSIFICATION": 시퀀스 분류, "TOKEN_CLASSIFICATION": 토큰 분류, "SEQ_2_SEQ_LM": 번역모델)
    task_type="CAUSAL_LM",

    target_modules=[
           "q_proj", "v_proj", "k_proj", "o_proj",
            "gate_proj", "up_proj", "down_proj"
    ]
)

training_params = TrainingArguments(
    # 학습 결과와 체크포인트를 저장할 디렉토리 경로
    output_dir= "./results",


    # 전체 데이터셋을 반복학습 주기 (1~50 정도 활용)
    num_train_epochs=5,
    # 각 GPU 장치에서 사용하는 배치 크기 (배치 크기에 따라 가중치 업데이트 1~64)
    # 배치크기는 모델히 한번에 처리하는 데이터 샘플의 수
    per_device_train_batch_size=1,
    #그래디언트 누적을 통한 배치 크기 증가 (여러 작은 배치를 합쳐서 효과적으로 큰 배치 크기를 적용 가능) *4 적용 시 그래디언트 누적 4개 후 한번에 가중치 업데이트
    # 배치크기가 4배 증가한 효과를 가지지만 실제 메모리 사용은 배치 1개 크기만 사용
    gradient_accumulation_steps=1,
    # 사용할 옵티마이저 설정 ("adamw, paged_adamw_32bit","adamw_torch")
    optim="paged_adamw_32bit",
    # 학습 중 모델 체크포인트 저장 주기 (스템 수)
    save_steps=25,
    # 학습 중 로그를 기록할 주기(스텝 수)
    logging_steps=25,
    # 학습률 설정 (모델의 파라미터 업데이트 시 스템 크기, 1e-6 ~ 1e-2))
    learning_rate=2e-4,
    # 모델의 가중치에 패널티 적용하여 과적합 방지 (0.0은 정규화 안함, 0.0 ~ 0.1)
    weight_decay=0.001,
    # 16비트 부동 소수점(FP16) 연산을 사용하여 메모리 사용량과 계산속도 개선 (사용시 정밀도 감소, 32비트 부동소수점보다 절반)
    fp16=False,
    # bfloat16 연산을 사용하여 학습 FP16대비 더 넓은 지수 범위 (사용시 정밀도 감소, 32비트 부동소수점보다 절반)
    bf16=False,
    # (그래디언트 클리핑을 위한 최대 노름 값 설정, 0.1 ~ 10)
    # 그래디언트 클리핑은 그래디언트의 크기가 너무 커서 학습이 불안정해지는 것을 방지하기 위한 기법 0.3 초과하지 않도록 함 (크면 가중치가 너무 크게업데이트됨)
    max_grad_norm=0.3,
    # 전체 학습단계 수 (-1 인 경우 epochs, 이외에는 스탭수,  -1~100000)
    max_steps=-1,
    # 학습률 warmup비율 설정 (학습 초기에 학습률을 서서히 증가시켜 안정적인 학습, 0.0~0.5)
    # 너무 높으면 학습률이 너무 늦어짐
    warmup_ratio=0.03,
    # 배치 내 시쿼스 길이를 그룹화하여 패딩을 최소화 (데이터 시권스길이 유사한것 끼리 그룹화하여 메모리 효율설 높임)
    group_by_length=True,
    # 학습률 스케줄러의 유형 설정 ("linear","cosine","constant","polynomial" 등) * constant는 학습률을 일정하게 유지
    lr_scheduler_type="constant",
    # 학습 로그를 보고할 플랫폼 설정 (아웃풋 디렉토리 참고) -> tensorboard --logdir=./results/runs * wandb로 설정 가능
    report_to="tensorboard"
)

"""# 6. Train Model"""

trainer = SFTTrainer(
    # 학습할 모델
    model=model,
    # 모델 학습에 사용할 데이터셋
    train_dataset=datasetCommon,
    #  PEFT(파라미터 효율적 미세 조정) 설정 정의
    peft_config=peft_params,
    # 데이터셋에서 학습 데이터셋 텍스트 필드 이름
    # dataset_text_field="text",
    # 입력 시퀀스 최대 길이 (128 ~ 1024) * 길이가 길수록 더많은 컨텍스트를 모델에 제공가능 단, 메모리 사용량 증가
    # max_seq_length=None,
    # 모델과 함께 사용할 토크나이저
    # tokenizer=tokenizer,
    args=training_params,
    # 입력 시퀀스 패킹여부 (패킹 시 짧고/긴 시퀀스를 혼합하여 배치 처리 서능 개선)
    # packing=False,
)
trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
It is strongly recommended to train Gemma3 models with the `eager` attention implementation instead of `sdpa`. Use `eager` with `AutoModelForCausalLM.from_pretrained('<path-to-checkpoint>', attn_implementation='eager')`.
C:\Users\HP\.finetune\Lib\site-packages\transformers\integrations\sdpa_attention.py:54: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Step,Training Loss
25,1.824400
50,1.308700
75,1.212300
100,1.195600
125,0.749800
150,0.880300
175,0.807400
200,0.809400
225,0.476900
250,0.534500


TrainOutput(global_step=500, training_loss=0.6543735513687133, metrics={'train_runtime': 383.4968, 'train_samples_per_second': 1.304, 'train_steps_per_second': 1.304, 'total_flos': 2973379152352800.0, 'train_loss': 0.6543735513687133})

In [22]:
now_str = datetime.now().strftime("%Y_%m_%d_%H")
print(now_str)

2025_07_18_16


In [23]:
if inColab == True:
    savePath = f"/content/gdrive/MyDrive/Colab Notebooks/models/gemma_tune_{now_str}"
    trainer.save_model(savePath)
else:
    savePath = f"./models/gemma_tune_{now_str}"
    trainer.save_model(savePath)